## Import SqliteCloudClient and SqliteCloudAccount

SqliteCloudAccount is the class rapresenting your auth data for SqliteCloud

SqliteCloudClient is the class managing the connection for you

In [1]:
from sqlitecloud.conn_info import user,password,host,db_name,port
from sqlitecloud.client import SqliteCloudClient, SqliteCloudAccount

Loading SQLITECLOUD lib from: /Users/sam/projects/codermine/sqlitecloud-sdk/C/libsqcloud.so


## Init a connection

Initialize the client with account connection info

In [2]:
account = SqliteCloudAccount(user, password, host, db_name, port)
client = SqliteCloudClient(cloud_account=account)
conn = client.open_connection()

In [3]:
db_name

'people'

Then execute the query

In [4]:
query = "select * from employees;"
result = client.exec_query(query, conn)

select * from employees;


The result is an iterable

In [5]:
for r in result:
    print(r)

{'emp_id': 1, 'emp_name': b'Bobby Tables'}
{'emp_id': 1, 'emp_name': b'Bobby Tables'}
{'emp_id': 1, 'emp_name': b'Bobby Tables'}
{'emp_id': 1, 'emp_name': b'Bobby Tables'}
{'emp_id': 1, 'emp_name': b'Bobby Tables'}
{'emp_id': 1, 'emp_name': b'Bobby Tables'}
{'emp_id': 1, 'emp_name': b'Bobby Tables'}
{'emp_id': 1, 'emp_name': b'Bobby Tables'}


Whe you are done clean up the connection

In [6]:
client.disconnect(conn)


You can bind values to parametric queries: you can pass parameters as positional values in an array

In [7]:
new_connection = client.open_connection()
result = client.exec_statement("select * from employees where emp_id = ? and emp_name = ? ", [1,'Bobby Tables'],conn=new_connection)
for r in result:
    print(r)
client.disconnect(conn)

ev type: <class 'int'>
ev type: <class 'str'>
